In [ ]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [31]:
#Importing the dataset
sdf = pd.read_csv(r'/workspace/share/jsondf.csv')

/tmp/ipykernel_348424/2948389231.py:3: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  sdf = pd.read_csv(r'/workspace/share/jsondf.csv')


In [69]:
#From int64 to str
sdf['id']=sdf['id'].astype(str)

#Opening the post_info.xslx file
info=pd.read_excel(r'post_info.xlsx', header=None)
info.columns=['num','owner.id','sponsored','id','pic']

#From col obj to str for compatibility
info['id']=info['id'].astype(str)

#For easy merging
info['id'] = info['id'].str.replace(r'.json', '')

#Merging
sdf = pd.DataFrame(pd.merge(sdf, info, how= 'inner', left_on=['id'], right_on=['id']))

sdf=sdf.dropna(subset=['comments_disabled'])
sdf=sdf.set_index(['id'])

KeyError: 'id'

In [38]:
sdf['pic_count']=0
for i in range(len(sdf)):
    sdf['pic_count'][i] = str(sdf['pic'][i]).count('.jpg')

sdf['hashtag_count']=0
for i in range(len(sdf)):
    sdf['hashtag_count'][i] = str(sdf['edge_media_to_caption'][i]).count('#')

sdf['tag_count']=0
for i in range(len(sdf)):
    sdf['tag_count'][i] = str(sdf['edge_media_to_caption'][i]).count('@')

/tmp/ipykernel_348424/1318166421.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sdf['pic_count'][i] = str(sdf['pic'][i]).count('.jpg')


In [41]:
sdf['ind']=range(len(sdf))
sdf=sdf.set_index(sdf['ind'])
sdf=sdf[sdf['edge_media_to_caption']!='[]']

sdf['edge_media_to_comment'] = sdf['edge_media_to_comment'].apply(lambda x: re.findall('\d+', x)[0])
sdf['edge_media_preview_like'] = sdf['edge_media_preview_like'].apply(lambda x: re.findall('\d+', x)[0])
sdf['edge_media_to_caption'] = sdf['edge_media_to_caption'].apply(lambda x: x[30:-5])

sdf = sdf.dropna(subset=['owner.id'])

In [45]:
influencerdf = pd.read_csv(r'/workspace/share/influencerdf.csv')

In [47]:
#Merging
ndf = pd.DataFrame(pd.merge(left=sdf, right=influencerdf, how='left', left_on=['owner.id'], right_on=['tag']))

ndf=ndf.dropna(subset=['comments_disabled','Posts'])
ndf['edge_media_to_caption']=ndf['edge_media_to_caption'].astype(str)

In [49]:
ndf.to_csv('ndf.csv')

In [50]:
ndf = pd.read_csv(r'/workspace/share/ndf.csv')

/tmp/ipykernel_348424/1617948190.py:2: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  ndf = pd.read_csv(r'/workspace/share/ndf.csv')


In [52]:
def replace_newline(string):
    modified_string = string.replace('\\n', ' ')
    return modified_string

In [54]:
brandsdf = pd.read_csv(r'/workspace/share/brandsdf.csv')
brandsdf=brandsdf.dropna(subset=['Category'])
brandsdf=brandsdf.drop(['ind'],axis=1)
brandsdf=brandsdf.drop(['ind.1'],axis=1)
brandsdf['ind']=range(len(brandsdf))
brandsdf=brandsdf.set_index(brandsdf['ind'])

#count=0
#while count<len(brandsdf):
  #brandsdf['tag'][count]= '@'+brandsdf['tag'][count]
  #count=count+1

brandsdf.to_csv(r'/workspace/share/brandsdf.csv', index = True)

In [62]:
ndf['ind']=range(len(ndf))
ndf=ndf.set_index(ndf['ind'])

In [63]:
def get_substrings_after(string, char):
    words = string.split()
    result = [word[word.index(char) + 1:] for word in words if char in word]
    return result

In [64]:
ndf['brands']=0
for i,rows in ndf.iterrows():
    ndf['brands'][i]=get_substrings_after(ndf['edge_media_to_caption'][i], '@')

ndf=ndf.filter(['edge_media_preview_like', 'edge_media_to_caption','edge_media_to_comment', 'is_ad','shortcode','taken_at_timestamp','owner.id','sponsored','pic', 'pic_count', 'hashtag_count', 'tag_count','Name', 'Followers', 'Followees', 'Posts','Category', 'Bio', 'E-mail', 'Phone','tag','brands'])

ndf.to_csv('ndf.csv')

/tmp/ipykernel_348424/3185902475.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ndf['brands'][i]=get_substrings_after(ndf['edge_media_to_caption'][i], '@')
IOStream.flush timed out


In [67]:
train_ndf, sample_thales = train_test_split(ndf, test_size=0.2, random_state=42)

sample_thales.to_csv('sample_thales.csv')